In [1]:
# from google.colab import drive
from pathlib import Path
# drive.mount('/content/gdrive', force_remount=True)
%cd "/content/drive/Othercomputers/my_mac/Coding_stuff/Projects/Surface_Predictor"

In [2]:
creds = '{"username":"phade160","key":"e780a54d20d6c7d34b674893c923068a"}'
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
  print('entered')
cred_path.parent.mkdir(exist_ok=True)
cred_path.write_text(creds)
cred_path.chmod(0o600)

In [3]:
# !kaggle competitions download -c career-con-2019 -p ./data/
# !unzip ./data/X_test.csv.zip -d ./data
# !unzip ./data/X_train.csv.zip -d ./data
# !rm ./data/X_train.csv.zip ./data/X_test.csv.zip

In [4]:
%%capture
%pip install -Uqq pytorch-lightning torchmetrics wandb

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import wandb
from tqdm.auto import tqdm 
from kaggle import api
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import WeightedRandomSampler

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import WandbLogger
import torchmetrics

from pylab import rcParams
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
%matplotlib inline
%config InlinBackend.figure_format = 'retina'
sns.set_style(style= 'whitegrid')
rcParams['figure.figsize'] = 14,9

In [7]:
pl.seed_everything(42)

42

In [8]:
!wandb login

In [9]:
x_train = pd.read_csv('./data/raw/X_train.csv', low_memory=False)
y_train = pd.read_csv('./data/raw/y_train.csv', low_memory=False)
x_test = pd.read_csv('./data/raw/X_test.csv', low_memory=False)

x_train

          row_id  series_id  ...  linear_acceleration_Y  linear_acceleration_Z
0            0_0          0  ...                 2.1030                -9.7532
1            0_1          0  ...                 1.5064                -9.4128
2            0_2          0  ...                 1.5922                -8.7267
3            0_3          0  ...                 1.0993               -10.0960
4            0_4          0  ...                 1.4689               -10.4410
...          ...        ...  ...                    ...                    ...
487675  3809_123       3809  ...                 2.0115                -9.0063
487676  3809_124       3809  ...                 3.0696                -8.1257
487677  3809_125       3809  ...                 4.2622                -8.1443
487678  3809_126       3809  ...                 4.7130                -9.4435
487679  3809_127       3809  ...                 4.2751               -10.4980

[487680 rows x 13 columns]

In [10]:
def load():
    x_train = pd.read_csv('./data/raw/X_train.csv', low_memory=False)
    y_train = pd.read_csv('./data/raw/y_train.csv', low_memory=False)
    x_test = pd.read_csv('./data/raw/X_test.csv', low_memory=False) 

    data = [x_train, y_train, x_test]
    return data
    

In [11]:
def load():
    x_train = pd.read_csv('./data/raw/X_train.csv', low_memory=False)
    y_train = pd.read_csv('./data/raw/y_train.csv', low_memory=False)
    x_test = pd.read_csv('./data/raw/X_test.csv', low_memory=False) 

    data = [[x_train, y_train], x_test]
    return data
    

In [12]:
def load():
    x_train = pd.read_csv('./data/raw/X_train.csv', low_memory=False)
    y_train = pd.read_csv('./data/raw/y_train.csv', low_memory=False)
    x_test = pd.read_csv('./data/raw/X_test.csv', low_memory=False) 

    data = [(x_train, y_train),(x_test)]
    return data
    

In [13]:
a,b = (1,)

In [14]:
a,b = (1, )

In [15]:
a,b = (1, [])

In [16]:
a,b = (1,)

In [17]:
a, = (1,)

In [18]:
a

1

In [19]:
a, = (1)

In [20]:
a = (1)

In [21]:
a

1

In [22]:
def load_and_log():
    with wandb.init(project='surface_pred', job_type='load-raw-data') as run:
        all_data = load()
        names = ['training', 'test']

        #create our artifact
        raw_data = wandb.Artifact('surface-raw', type='dataset',
                                 description = 'Raw dataset containin x_train and y_train for the training, and x_test for the testing',
                                 metadata = {'source': 'kaggle.com/career-con-2019',
                                             'sizes': [len(data[0]) for data in all_data]
                                             })

        for name, data in zip(names, all_data):
            #store a new file to the created artifact
            with raw_data.new_file(name + '.pt', mode='wb') as file:
                if name == 'training':
                    x, y = data
                    torch.save((x,y), file)
                else:
                    x, = data
                    torch.save((x), file)

        run.log_artifact(raw_data)

load_and_log()

In [23]:
def load():
    x_train = pd.read_csv('./data/raw/X_train.csv', low_memory=False)
    y_train = pd.read_csv('./data/raw/y_train.csv', low_memory=False)
    x_test = pd.read_csv('./data/raw/X_test.csv', low_memory=False) 

    data = [(x_train, y_train),(x_test, )]
    return data
    

In [24]:
def load_and_log():
    with wandb.init(project='surface_pred', job_type='load-raw-data') as run:
        all_data = load()
        names = ['training', 'test']

        #create our artifact
        raw_data = wandb.Artifact('surface-raw', type='dataset',
                                 description = 'Raw dataset containin x_train and y_train for the training, and x_test for the testing',
                                 metadata = {'source': 'kaggle.com/career-con-2019',
                                             'sizes': [len(data[0]) for data in all_data]
                                             })

        for name, data in zip(names, all_data):
            #store a new file to the created artifact
            with raw_data.new_file(name + '.pt', mode='wb') as file:
                if name == 'training':
                    x, y = data
                    torch.save((x,y), file)
                else:
                    x, = data
                    torch.save((x), file)

        run.log_artifact(raw_data)

load_and_log()

In [25]:

run = wandb.init()
artifact = run.use_artifact('phade160/surface_pred/surface-raw:v0', type='dataset')
artifact_dir = artifact.download()

In [26]:
artifact_dir

'./artifacts/surface-raw:v0'

In [27]:
import os

In [28]:
os.path.join(artifact_dir, 'training.pt')

'./artifacts/surface-raw:v0/training.pt'

In [29]:
torch.load(os.path.join(artifact_dir, 'training.pt'))

(          row_id  series_id  ...  linear_acceleration_Y  linear_acceleration_Z
 0            0_0          0  ...                 2.1030                -9.7532
 1            0_1          0  ...                 1.5064                -9.4128
 2            0_2          0  ...                 1.5922                -8.7267
 3            0_3          0  ...                 1.0993               -10.0960
 4            0_4          0  ...                 1.4689               -10.4410
 ...          ...        ...  ...                    ...                    ...
 487675  3809_123       3809  ...                 2.0115                -9.0063
 487676  3809_124       3809  ...                 3.0696                -8.1257
 487677  3809_125       3809  ...                 4.2622                -8.1443
 487678  3809_126       3809  ...                 4.7130                -9.4435
 487679  3809_127       3809  ...                 4.2751               -10.4980
 
 [487680 rows x 13 columns],       ser

In [30]:
def get_sequences(x, y=None): 
  sequences = []
  for series_id, group in x.groupby('series_id'):
    sequence_features = group[feature_columns]
    if y:
        label = y[y.series_id == series_id].iloc[0].label
        sequences.append((sequence_features, label))
      
    else:
        sequences.append(sequence_features)

  return sequences

In [31]:
torch.load(os.path.join(artifact_dir, 'test.pt'))

          row_id  series_id  ...  linear_acceleration_Y  linear_acceleration_Z
0            0_0          0  ...                 2.8027                -9.6816
1            0_1          0  ...                 2.5408                -9.8521
2            0_2          0  ...                 2.5853                -9.3835
3            0_3          0  ...                 2.9966                -8.7415
4            0_4          0  ...                 2.6498                -8.8432
...          ...        ...  ...                    ...                    ...
488443  3815_123       3815  ...                 3.5421                -8.4445
488444  3815_124       3815  ...                 3.3380                -8.8012
488445  3815_125       3815  ...                 3.2110                -9.3700
488446  3815_126       3815  ...                 2.8634                -9.8546
488447  3815_127       3815  ...                 2.5159               -10.5870

[488448 rows x 13 columns]

In [32]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x)
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences)

                
        

In [33]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x)
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences)

        run.log_artifact(preprocessed_data)

                
        

In [34]:
preprocess_and_log()

In [35]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x, = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x)
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences)

        run.log_artifact(preprocessed_data)

                
        

In [36]:
preprocess_and_log()

In [37]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x)
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences)

        run.log_artifact(preprocessed_data)

                
        

In [38]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x[0])
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences)

        run.log_artifact(preprocessed_data)

                
        

In [39]:
preprocess_and_log()

In [40]:
#we dont need the first 3 cols
feature_columns = x_train.columns.to_list()[3:]
feature_columns

['orientation_X',
 'orientation_Y',
 'orientation_Z',
 'orientation_W',
 'angular_velocity_X',
 'angular_velocity_Y',
 'angular_velocity_Z',
 'linear_acceleration_X',
 'linear_acceleration_Y',
 'linear_acceleration_Z']

In [41]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x[0])
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences)

        run.log_artifact(preprocessed_data)

                
        

In [42]:
preprocess_and_log()

In [43]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x[0])
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences, file)

        run.log_artifact(preprocessed_data)

                
        

In [44]:
preprocess_and_log()

In [45]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x)
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences, file)

        run.log_artifact(preprocessed_data)

                
        

In [46]:
preprocess_and_log()

In [47]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'taining':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                print(x)
                test_sequences = get_sequences(x)
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences, file)

        run.log_artifact(preprocessed_data)

                
        

In [48]:
preprocess_and_log()

In [49]:
def preprocess_and_log():
    with wandb.init(project='surface_pred', job_type='preprocess-data') as run:
        preprocessed_data = wandb.Artifact(name='surface-preprocess', type='dataset',
                                            description='get sequences from raw data and split train sequences to train val',
                                            metadata={'get_sequences': True,
                                                      'split_train' : True})
        # Declare which artifact to be used
        raw_data_artifact = run.use_artifact('surface-raw:latest')
        #download it
        raw_data = raw_data_artifact.download()
        
        for data in ['training', 'test']:
            filename = data + '.pt'
            if data == 'training':
                x, y = torch.load(os.path.join(raw_data, filename))
                sequences = get_sequences(x,y)
                train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
            else:
                x = torch.load(os.path.join(raw_data, filename))
                test_sequences = get_sequences(x)
            with preprocessed_data.new_file(data + '.pt', mode='wb') as file:
                if data == 'taining':
                    torch.save((train_sequences, val_sequences), file)

                else:
                    torch.save(test_sequences, file)

        run.log_artifact(preprocessed_data)

                
        

In [50]:
preprocess_and_log()